In [1]:
import numpy as np
import torch

from sklearn.preprocessing import MinMaxScaler
from deep_traffic_generation.core.transforms import PyTMinMaxScaler

In [2]:
X = np.array([
    [i, i, 0.0] for i in range(5)
])
X

/home/adrien/.conda/envs/traffic/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[0., 0., 0.],
       [1., 1., 0.],
       [2., 2., 0.],
       [3., 3., 0.],
       [4., 4., 0.]])

In [3]:
nav = np.array([
    [i, i+2] for i in range(2)
])
nav

array([[0, 2],
       [1, 3]])

In [4]:
x1 = X[:, 0, np.newaxis]
y1 = X[:, 1, np.newaxis]
tracks = X[:, 2, np.newaxis]

x2 = nav[:, 0]
y2 = nav[:, 1]

In [15]:
bearings = np.arctan2(x2-x1,y2-y1) * 180 / np.pi
bearings

array([[ 0.        ,  0.32175055],
       [-0.78539816,  0.        ],
       [-1.57079633, -0.78539816],
       [-1.89254688, -1.57079633],
       [-2.03444394, -1.89254688]])

In [6]:
np.argmin(abs(tracks-bearings), axis=1)

array([0, 1, 1, 1, 1])

In [7]:
np.min(abs(tracks-bearings), axis=1)

array([  0.        ,   0.        ,  45.        ,  90.        ,
       108.43494882])

_________________

In [8]:
scaler = MinMaxScaler(feature_range=(-1, 1)).fit(X)
X_scaled = scaler.transform(X)
X_scaled

array([[-1. , -1. , -1. ],
       [-0.5, -0.5, -1. ],
       [ 0. ,  0. , -1. ],
       [ 0.5,  0.5, -1. ],
       [ 1. ,  1. , -1. ]])

In [9]:
X_t = torch.from_numpy(X)
X_t

/home/adrien/.conda/envs/traffic/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([[0., 0., 0.],
        [1., 1., 0.],
        [2., 2., 0.],
        [3., 3., 0.],
        [4., 4., 0.]], dtype=torch.float64)

In [10]:
X_t.requires_grad_()
scaler = PyTMinMaxScaler(feature_range=(-1, 1)).fit(X_t)
X_t = scaler.transform(X_t)
X_t

tensor([[-1.0000, -1.0000, -1.0000],
        [-0.5000, -0.5000, -1.0000],
        [ 0.0000,  0.0000, -1.0000],
        [ 0.5000,  0.5000, -1.0000],
        [ 1.0000,  1.0000, -1.0000]], dtype=torch.float64,
       grad_fn=<AddBackward0>)

In [11]:
scaler.inverse_transform(X_t)

tensor([[0., 0., 0.],
        [1., 1., 0.],
        [2., 2., 0.],
        [3., 3., 0.],
        [4., 4., 0.]], dtype=torch.float64, grad_fn=<DivBackward0>)

In [12]:
scaler.partial_transform(torch.from_numpy(nav), idxs=np.where(np.isin(["x", "y", "track"], ["x", "y"]))[0])

tensor([[-1.0000,  0.0000],
        [-0.5000,  0.5000]], dtype=torch.float64, grad_fn=<AddBackward0>)

In [13]:
X_t[:, 2] = scaler.partial_inverse(X_t[:, 2], idxs=2)
X_t

tensor([[-1.0000, -1.0000,  0.0000],
        [-0.5000, -0.5000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.5000,  0.5000,  0.0000],
        [ 1.0000,  1.0000,  0.0000]], dtype=torch.float64,
       grad_fn=<CopySlices>)

In [14]:
X_t

tensor([[-1.0000, -1.0000,  0.0000],
        [-0.5000, -0.5000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.5000,  0.5000,  0.0000],
        [ 1.0000,  1.0000,  0.0000]], dtype=torch.float64,
       grad_fn=<CopySlices>)